Урок 9.

Задание
Разобраться с моделью генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов

In [1]:
!pip install stop_words

  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32895 sha256=5ca47f239a97e5d1c6c675b3321d1130e5cec45ff4db4ebcbcabe2a2adff71b0
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop_words


In [2]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 25.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=dd61e4aa969158f79b93615e76156f6670678ea05fb5c4e3d2196ca8245f0f4f
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [3]:

import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
text = open('/content/drive/MyDrive/nlp/les9/mt.txt', 'rb').read()

In [12]:
text=text.decode(encoding='utf-8')

In [13]:
print(text[:500])

﻿

Михаил Юрьевич Лермонтов

Мцыри[1 - Мцыри на грузинском языке значит «неслужащий монах», нечто вроде «послушника». (Примечание Лермонтова)]

Вкушая, вкусих мало меда и се аз умираю.

    1-я Книга царств.



1

		Немного лет тому назад,
		Там, где сливаяся шумят
		Обнявшись, будто две сестры,
		Струи Арагвы и Куры,
		Был монастырь. Из-за горы
		И нынче видит пешеход
		Столбы обрушенных ворот,
		И башни, и церковный свод;
		Но не курится уж под ним
		Кадильниц благовонны


In [ ]:
#text = text + text

In [14]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

121 unique characters


In [15]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [16]:
text_as_int, text[:2000], len(text_as_int), len(text)

(array([120,   2,   1, ...,   1,   2,   1]),
 '\ufeff\r\n\r\nМихаил Юрьевич Лермонтов\r\n\r\nМцыри[1 - Мцыри на грузинском языке значит «неслужащий монах», нечто вроде «послушника». (Примечание Лермонтова)]\r\n\r\nВкушая, вкусих мало меда и се аз умираю.\r\n\r\n\xa0\xa0\xa0\xa01-я Книга царств.\r\n\r\n\r\n\r\n1\r\n\r\n\t\tНемного лет тому назад,\r\n\t\tТам, где сливаяся шумят\r\n\t\tОбнявшись, будто две сестры,\r\n\t\tСтруи Арагвы и Куры,\r\n\t\tБыл монастырь. Из-за горы\r\n\t\tИ нынче видит пешеход\r\n\t\tСтолбы обрушенных ворот,\r\n\t\tИ башни, и церковный свод;\r\n\t\tНо не курится уж под ним\r\n\t\tКадильниц благовонный дым,\r\n\t\tНе слышно пенье в поздний час\r\n\t\tМолящих иноков за нас.\r\n\t\tТеперь один старик седой,\r\n\t\tРазвалин страж полуживой,\r\n\t\tЛюдьми и смертию забыт,\r\n\t\tСметает пыль с могильных плит,\r\n\t\tКоторых надпись говорит\r\n\t\tО славе прошлой – и о том,\r\n\t\tКак удручен своим венцом,\r\n\t\tТакой-то царь, в такой-то год,\r\n\t\tВручал России свой

In [17]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

﻿








In [18]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'\ufeff\r\n\r\nМихаил Юрьевич Лермонтов\r\n\r\nМцыри[1 - Мцыри на грузинском языке значит «неслужащий монах», нечто'
' вроде «послушника». (Примечание Лермонтова)]\r\n\r\nВкушая, вкусих мало меда и се аз умираю.\r\n\r\n\xa0\xa0\xa0\xa01-я '
'Книга царств.\r\n\r\n\r\n\r\n1\r\n\r\n\t\tНемного лет тому назад,\r\n\t\tТам, где сливаяся шумят\r\n\t\tОбнявшись, будто дв'
'е сестры,\r\n\t\tСтруи Арагвы и Куры,\r\n\t\tБыл монастырь. Из-за горы\r\n\t\tИ нынче видит пешеход\r\n\t\tСтолбы обр'
'ушенных ворот,\r\n\t\tИ башни, и церковный свод;\r\n\t\tНо не курится уж под ним\r\n\t\tКадильниц благовонный дым'


In [19]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [20]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\ufeff\r\n\r\nМихаил Юрьевич Лермонтов\r\n\r\nМцыри[1 - Мцыри на грузинском языке значит «неслужащий монах», нечт'
Target data: '\r\n\r\nМихаил Юрьевич Лермонтов\r\n\r\nМцыри[1 - Мцыри на грузинском языке значит «неслужащий монах», нечто'


In [21]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [22]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [23]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [24]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [25]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 121) # (batch_size, sequence_length, vocab_size)


In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           30976     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dense (Dense)               (64, None, 121)           124025    
                                                                 
Total params: 30580089 (116.65 MB)
Trainable params: 305

In [27]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 121), dtype=float32, numpy=
array([[ 5.8831092e-06, -1.2645363e-05, -1.6718321e-05, ...,
         4.6452360e-06, -4.2702950e-06,  8.9548867e-06],
       [ 5.1073588e-05, -3.6334299e-05, -4.0042676e-05, ...,
         2.3019729e-05, -2.0806256e-06,  1.2431354e-05],
       [ 1.4571410e-04, -6.6178167e-05, -7.3943971e-05, ...,
         3.9565388e-05,  8.9681816e-06,  2.1601023e-05],
       ...,
       [ 9.5381949e-04,  9.3608431e-04, -9.7719312e-04, ...,
        -6.0852035e-04,  8.2521664e-04, -1.1078044e-04],
       [ 7.2732946e-04,  7.3790574e-04, -8.2731387e-04, ...,
        -4.1764288e-04,  1.0915911e-03,  6.0959137e-06],
       [ 4.6830106e-04,  5.1998615e-04, -6.7025423e-04, ...,
        -2.2570157e-04,  1.2992260e-03,  1.3889454e-04]], dtype=float32)>

In [28]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [29]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 ' была,\r\n\t\tНо в мысль она мне залегла,\r\n\t\tИ мне, лишь сумрак настает,\r\n\t\tНезримый дух ее поет.\r\n\r\n\r\n\r'

Next Char Predictions: 
 '7ж ;юЮШШуIёкЦДцrКc6д)8…-—\xa0mIiжКщv„a8lШb4нЦ44(vоБ9гЯмuЗ\ufeffa6h1s2sчuЕ»фчльОд1Зз»V„еXXюXшЦ,tдn] _яз_ьnШrу'


In [30]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 121)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.795726


In [31]:
model.compile(optimizer='adam', loss=loss)

In [32]:
!rm -rf ./training_checkpoints

In [33]:
!ls ./training_checkpoints

ls: cannot access './training_checkpoints': No such file or directory


In [34]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=7*10,
    save_weights_only=True)

In [35]:
EPOCHS = 100
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
4/4 [==============================] - 14s 352ms/step - loss: 4.5779
Epoch 2/100
4/4 [==============================] - 2s 398ms/step - loss: 4.0560
Epoch 3/100
4/4 [==============================] - 2s 381ms/step - loss: 3.6299
Epoch 4/100
4/4 [==============================] - 2s 336ms/step - loss: 3.5454
Epoch 5/100
4/4 [==============================] - 1s 338ms/step - loss: 3.5375
Epoch 6/100
4/4 [==============================] - 1s 336ms/step - loss: 3.5373
Epoch 7/100
4/4 [==============================] - 1s 339ms/step - loss: 3.5353
Epoch 8/100
4/4 [==============================] - 1s 339ms/step - loss: 3.5305
Epoch 9/100
4/4 [==============================] - 1s 341ms/step - loss: 3.5325
Epoch 10/100
4/4 [==============================] - 1s 340ms/step - loss: 3.5331
Epoch 11/100
4/4 [==============================] - 1s 344ms/step - loss: 3.5330
Epoch 12/100
4/4 [==============================] - 1s 342ms/step - loss: 3.5332
Epoch 13/100
4/4 [==================

In [36]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_88'

In [37]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [38]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            30976     
                                                                 
 lstm_4 (LSTM)               (1, None, 1024)           5246976   
                                                                 
 lstm_5 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_6 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_7 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 dense_1 (Dense)             (1, None, 121)            124025    
                                                                 
Total params: 30580089 (116.65 MB)
Trainable params: 3

In [39]:
def generate_text(model, start_string, temperature=1, num_generate=500):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = num_generate

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = temperature

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

Продемонстрируем, как обучилась сеть. Возьмем достаточно большую температуру (подобрана эмпирически):

In [40]:
text_ = generate_text(model, start_string=u"Прекрасны горы под луной ", temperature=1.3)
print(text_)

		Бэыскнлте ырыялбоое бу6» ня,
		Ва хжирьсыю– шео! икбткыек8та т5..		1рокемё
!И1ЯлЯни Хы мсетолй тдыя натрао»трплрезя
		Вол лауй1гскн ыьлАобдст.
		Оввв хинели.
	
 oСдннрабтжы ;хыее–лО оyрруря,

	Оо тtьвисБдо ееиaРсжоингасей…
		гронмечотя еежосжсш
 яакоснркассыщкыанй ыеллы.
/
В зпмдпбйкрыоа,

Я
Иойпспынгуе я.


	–цдемвосвыис ааздсоквяКи встаим
Т жеткбги олаккбялворем

	14лтиабет0нрпня
		Мвсаий

	–1ПБцоахриво;


	Я гау_л вйатт т мхсквя2Ц


Далее наиболее удачная температура для генерации - при ней модель соскакивает с одного зацикливания на другое:

In [41]:
text_ = generate_text(model, start_string=u"Прекрасны горы под луной ", temperature=1.1)
print(text_)

Прекрасны горы под луной мтй
	
Зцоо. я 0 оечк  ьнк:


		стаоеи ншрасоеу…

	Тба оиагио“
Я	Пнх ныкниаатыоошынодам бкнх!

	Ня ыа ру-цыы геiа8:,нМОетзвьнлшыагетненьбоаоу.рСтеыэкай!
		9. нбвГоя

		ъе? 0 окыo2П. Мсгсщыи-а4,

	аКыллни, ечжеелооущо тиа…
		Нтельснанйлтей» соое пвившш1 спстl ностмя

	Вка соои длбкошолеъ1нкйя,Сотрзывенстивоч овсггоышдоо зил.



2НИ оркоысиаимбоагези снтй :.
		Срилеюдчивцйжсоаа1
Н“ХЯ втбенбй уекла жынтттум  ниам(Н…
		Ппллть оой цооки обнаа соо вX
		оБ


In [42]:
text_ = generate_text(model, start_string=u"Прекрасны горы под луной ", temperature=.1)
print(text_)

Прекрасны горы под луной и ооо оое оо оое ооо оое оо оо оо оо оо оо и ооо оо оо оо оо оо ооно и ое оо оо ооо оо оо оо оо оо оо оо оо оо ое оо оо оото оо ое оо ое оо оо ое оо оо ооо оо оо оо ооо и оо оо ооо ооо оо ооо оо оо ооо и ооо оо ооо оо оо ое ио ооо оо оо ооо оое ио ооо оо оое оо ооо оо оо оое ооо оо ооо ооо оо оое оое ооно и оо оо оо оо ое ое ооо ооно оо ооо оо оое оо ое оо оо ое ое оо оо оо ооо ое оо ооно оо ооно оо ооо оо оо оо ооо оо оо оо оо ое оо ооо оое оо оо оо ое оо оо оо оо оо оо оо ое оо ооо оо оо ооно 


In [ ]:
text_ = generate_text(model, start_string=u"Прекрасны горы под луной ", temperature=.01)
print(text_)

Прекрасны горы под луной дад коста,
     И толь в прокладных страных стенах
     Душой дитя, судьбой монах.
     Я никому не мог сказать
     Священных слов «отец» и «мать».
     Конечно, ты хотел, старик,
     Чтоб я в обители отвык
     От этих сладостных имен, –
     Напрасно: звук их был рожден
     Со мной. И видел у других
     Отчизну, дом, друзей, родных,
     А у себя не находил
     Не только милых душ – могил!
     Тогда, пустых не тратя слез,
     В душе я клятву произнес:
     Хотя на миг ког


In [43]:
len(text_)

525

**Выводы **
- Модель воспроизводит длинную последовательность.
- Модель вместо составления смешных предложений из отдельных слов сваливается в воспроизведение нескольких или одного отрывка из произведения.